<a href="https://colab.research.google.com/github/olgasherbiena/KPI/blob/main/%D0%90%D0%9A_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install kagglehub


In [26]:
import pandas as pd
import kagglehub
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

csv_path  = kagglehub.dataset_download("shantanudhakadd/email-spam-detection-dataset-classification")
csv_file = os.path.join(csv_path, 'spam.csv')
df = pd.read_csv(csv_file, encoding='latin1')

Using Colab cache for faster access to the 'email-spam-detection-dataset-classification' dataset.


In [27]:
df = df[['v1', 'v2']].copy()
df.columns = ['label', 'text']
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

print(df.head())
print(df['label'].value_counts())

   label                                               text
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...
label
0    4825
1     747
Name: count, dtype: int64


In [28]:
import numpy as np

class LSH:
    def __init__(self, num_buckets, num_hash_functions, input_dim):
        self.num_buckets = num_buckets
        self.num_hash_functions = num_hash_functions
        self.input_dim = input_dim
        self.hash_functions = [self._generate_hash_function() for _ in range(num_hash_functions)]
        self.buckets = {i: [] for i in range(num_buckets)}

    def _generate_hash_function(self):
        return np.random.randn(self.input_dim)

    def hash(self, vector):
        hashes = []
        for hash_function in self.hash_functions:
            hash_value = np.dot(vector, hash_function)
            hash_value = hash_value / np.linalg.norm(hash_function)
            hash_value = hash_value.astype(int)
            hashes.append(hash_value)
        return hashes

    def add_vector(self, vector):
        hashes = self.hash(vector)
        for i, hash_value in enumerate(hashes):
            self.buckets[hash_value % self.num_buckets].append((i, vector))

    def query(self, query_vector):
        query_hashes = self.hash(query_vector)
        similar_items = []
        for hash_value in query_hashes:
            bucket = self.buckets[hash_value % self.num_buckets]
            for item in bucket:
                if np.array_equal(item[1], query_vector):
                    continue
                if not any(np.array_equal(item[1], sim[1]) for sim in similar_items):
                    similar_items.append(item)
        return similar_items


In [29]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['text'])  # 'text' — колонка з email
X_dense = X.toarray()  # перетворюємо в звичайний numpy array

In [ ]:
lsh = LSH(num_buckets=10, num_hash_functions=5, input_dim=X_dense.shape[1])
for vec in X_dense:
    lsh.add_vector(vec)
query_vec = X_dense[0]
similar_items = lsh.query(query_vec)
print("Схожі об'єкти:", len(similar_items))